# Convolutional networks
The objective of this worksheet is to introduce you to a specific type of layer: the convolutional layer.
Neural networks that contain such layers are typically called 'Convolutional networks'.

- 'convnets' for short, are networks with their initial layers based on the operation of convolution
- convolutional layers have several effects
 - they improve the statistical efficiency through parameter sharing a.k.a. tied weights (a regularization effect)
 - they can be seen as filters of the input that pick out a specific feature over the entire input
 - main uses: image and signal processing, recently also for text processing
- conv layers are often coupled together with aggregating layers to reduce the number of neurons in subsequent layers -> max pooling


1. Time this code (per-epoch) on your computational device, on the zseason computer, it takes 75s/epoch.  We don't like that.
2. Can you convert this to GPU-based code? How fast can you make it run?
3. What is the lowest test error you can achieve?

In [ ]:
require 'torch'
require 'nn'
require 'utils'
require 'optim'
print('ready')

In [ ]:
-- snippet shamelessly ripped from https://github.com/torch/tutorials/blob/master/A_datasets/mnist.lua 
-- load the training data (downloads when first used)
tar = 'http://torch7.s3-website-us-east-1.amazonaws.com/data/mnist.t7.tgz'

if not paths.dirp('mnist.t7') then
   print('==> downloading dataset')
   os.execute('wget ' .. tar)
   os.execute('tar xvf ' .. paths.basename(tar))
end

train_file = 'mnist.t7/train_32x32.t7'
test_file = 'mnist.t7/test_32x32.t7'

----------------------------------------------------------------------
print('==> loading dataset')

-- We load the dataset from disk, it's straightforward

train_data = torch.load(train_file,'ascii')
test_data = torch.load(test_file,'ascii')

In [ ]:
-- let's preprocess the data a bit, so they are between 0 and 1
train_data.data = train_data.data:type('torch.DoubleTensor'):mul(1.0 / 256.0)
test_data.data = test_data.data:type('torch.DoubleTensor'):mul(1.0 / 256.0)

In [ ]:
img = test_data.data[{519}]:clone()
itorch.image(img)
itorch.image(utils.enlarge(img,4))

In [ ]:
?nn.SpatialConvolution

In [ ]:
conv2 = nn.SpatialConvolution(1, 1, 2, 2)
print(conv2)
conv2.weight:zero()
conv2.weight[{1,1,1,1}] = -1
conv2.weight[{1,1,2,2}] = 1
conv2.bias[1] = 0

In [ ]:
-- let's print out only the last two dimensions
print(conv2.weight[{1,1}])

In [ ]:
-- what does this filter do?
itorch.image(utils.enlarge(conv2:forward(img),4))

## Max-Pooling
- reducing the amount of processed data
- enforcing translational invariance

In [ ]:
?nn.SpatialMaxPooling

In [ ]:
mp = nn.SpatialMaxPooling(2, 2, 2, 2)
seq = nn.Sequential()
seq:add(conv2)
seq:add(mp)

In [ ]:
itorch.image(utils.enlarge(seq:forward(img), 4))

## Multiple filters at the same time - input/output planes
- planes allow us to keep the networks sequential while being able to apply multiple filters
- we **stack** filters on top of each other to generate multiple output images

In [ ]:
conv2_2 = nn.SpatialConvolution(1, 2, 2, 2)
conv2_2.weight:zero()
print(conv2_2.weight:size())
conv2_2.weight[{1,1,{1,2},1}] = 1
conv2_2.weight[{1,1,{1,2},2}] = -1
conv2_2.weight[{2,1,1,{1,2}}] = 1
conv2_2.weight[{2,1,2,{1,2}}] = -1

In [ ]:
img2_2 = conv2_2:forward(img)
print(img2_2:size())

In [ ]:
-- iTorch already knows, how to handle stacks of images
itorch.image(utils.enlarge(img2_2,4))

In [ ]:
?nn.Reshape

In [ ]:
?nn.SpatialMaxPooling

## Convolutional network to process MNIST data

In [ ]:
-- network structure taken from 
-- http://nbviewer.ipython.org/github/eladhoffer/Talks/blob/master/DL_class2015/Deep%20Learning%20with%20Torch.ipynb
net = nn.Sequential()
net:add(nn.SpatialConvolution(1, 8, 5, 5)) -- 1 input image channel, 64 output channels, 5x5 convolution kernel
net:add(nn.SpatialMaxPooling(2,2,2,2))      -- A max-pooling operation that looks at 2x2 windows and finds the max.
net:add(nn.ReLU())                          -- ReLU activation function
net:add(nn.SpatialConvolution(8, 16, 3, 3))
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(16, 32, 3, 3))
net:add(nn.View(32*4*4):setNumInputDims(3)) -- reshapes from a 3D tensor of 32x4x4 into 1D tensor of 32*4*4
net:add(nn.Linear(32*4*4, 128))             -- fully connected layer (matrix multiplication between input and weights)
net:add(nn.ReLU())
net:add(nn.Dropout(0.5))                    --Dropout layer with p=0.5
net:add(nn.Linear(128,10))                  -- 10 is the number of outputs of the network (in this case, 10 digits)

-- loss is cross-entropy (== logsoftmax + classnllcriterion)
loss = nn.CrossEntropyCriterion()
print(net)

parameters, gradParameters = net:getParameters()

In [ ]:
-- random initialization according to Xavier
local tmp = math.sqrt(1. / 25)
net:get(1).weight:uniform(-tmp, tmp)
net:get(1).bias:zero()
tmp = math.sqrt(1. / 9)
net:get(4).weight:uniform(-tmp, tmp)
net:get(4).bias:zero()
tmp = math.sqrt(1. / 9)
net:get(7).weight:uniform(-tmp, tmp)
net:get(7).bias:zero()

tmp = math.sqrt(1. / net:get(9).bias:size(1))
net:get(9).weight:uniform(-tmp, tmp)
net:get(9).bias:zero()
tmp = math.sqrt(1. / net:get(12).bias:size(1))
net:get(12).weight:uniform(-tmp, tmp)
net:get(12).bias:zero()

-- initialize state for the optimizer
opt_state = {}
print('Random init complete')

In [ ]:
-- optimization parameters
nepochs = 50
ntrain = 60000
--ntrain = 10000
ntest = 10000
batch_size = 100
etest = 1

-- SGD
--opt_param = {
--    learningRate = 0.01,
--    momentum = 0.9,
--    learningRateDecay = 1e-3
--}

-- AdaDelta
execute_optimizer = optim.adadelta
opt_params = {
    rho = 0.9,
    eps = 1e-6
}

-- train the network
for e=1,nepochs do
    
    local timer = torch.Timer() 
    local confmat = optim.ConfusionMatrix(10, {'0','1','2','3','4','5','6','7','8','9'})
    local train_err = 0
    for b=1,ntrain/batch_size do
        
        function batch_eval(x)
            
            local err = 0
            
            if x ~= parameters then
                parameters:copy(x)
            end
            
            -- DO NOT FORGET TO ZERO THE GRADIENT STORAGE
            -- BEFORE EACH MINIBATCH
            gradParameters:zero()
        
            for i=1,batch_size do
                -- select current training example
                local ndx = (b-1) * batch_size + i
                local x = train_data.data[ndx]
                local t = train_data.labels[ndx]
            
                -- run it through the network & accumulate error
                local y = net:forward(x)
                err = err + loss:forward(y, t)
                
                confmat:add(y, t)

                -- run it backward from the loss criterion
                local dt_dy = loss:backward(y, t) 
                net:backward(x, dt_dy)
            end
            
            train_err = train_err + err
            
            return err, gradParameters
        end
        
        execute_optimizer(batch_eval, parameters, opt_params, opt_state)
    end
    
    print('******************* EPOCH ' .. e .. ' ************************')

    print('TRAINING [error = ' .. train_err .. ']')
    print(confmat:__tostring__())
    
    -- compute testing error every etest epochs
    if e % etest == 0 then
        local terr = 0
        local p = 0
        confmat:zero()
        for i=1,ntest do
            local x = test_data.data[i]
            local t = test_data.labels[i]
            local y = net:forward(x)
            confmat:add(y, t)
            terr = terr + loss:forward(y, t)
        end
        print('TESTING [error = ' .. terr .. ']')
        print(confmat:__tostring__())
    end
        
    print('Epoch took ' .. timer:time().real .. ' seconds.')
end

In [ ]:
-- Let us pass the image through the first convolutional layer and look at the results
imgf = net:get(1):forward(img)
print('Source input')
itorch.image(img)
print('After convolution')
itorch.image(image.scale(image.toDisplayTensor({input=imgf}),400))
print('Convolutional filters')
scaled_weights = image.scale(image.toDisplayTensor({input=net:get(1).weight,padding=2}),300)
itorch.image(scaled_weights)